# Usage estimation for V3 queries

<b>Table of Contents</b>

- Overview
- Setup
    - Get authentication token
- Query to estimate usage
    - Understanding the outputs

## Overview

In this section, we leverage the `dryRun` functionality of the Geospatial API to estimate the usage in bytes for a V3 query, based on the specified spatial and temporal ranges. By leveraging this feature, users can get a better understanding of resource consumption before actually running the queries.


## Setup

Before starting, you have to install IBM EI Geospatial python SDK i.e ibmpairs and other modules needed to run this sample, mentioned in the below cell.

<b>Note:</b> To run this notebook seamlessly, you have to first configure your Geospatial APIs' credentials in a file named `secrets.ini` in the below format:

```
[EI]
api.api_key = <Your Geospatial APIs API key>
api.tenant_id = <Your Geospatial APIs Tenant ID>
api.org_id = <Your Geospatial APIs Org ID>

```
Keep the secrets.ini file at an appropriate relative location of this notebook. For example, as specified in the config below.

```config.read('../../../auth/secrets.ini')```

In [ ]:
%pip install ibmpairs
%pip install configparser

In [13]:
import ibmpairs.client as client
import ibmpairs.query as query
import configparser

### Get authentication token

Now, let's create a client object using the API_KEY, TENANT_ID (or CLIENT_ID) and ORG_ID to create an authenticated HTTP client Authentication token using 'ibmpairs.client' module, to use it in the subsequent steps.

In [14]:
config = configparser.RawConfigParser()
config.read('../../../auth/secrets.ini')

EI_ORG_ID     = config.get('EI', 'api.org_id') 
EI_TENANT_ID  = config.get('EI', 'api.tenant_id') 
EI_APIKEY     = config.get('EI', 'api.api_key')

EI_client_v3 = client.get_client(org_id    = EI_ORG_ID,
                                  tenant_id = EI_TENANT_ID,
                                  api_key     = EI_APIKEY,
                                  version   = 3
                                )

2025-03-20 13:37:26 - paw - INFO - The client authentication method is assumed to be OAuth2.
2025-03-20 13:37:26 - paw - INFO - Legacy Environment is False
2025-03-20 13:37:26 - paw - INFO - The authentication api key type is assumed to be IBM EIS, because the api key prefix 'PHX' is present.
2025-03-20 13:37:29 - paw - INFO - Authentication success.
2025-03-20 13:37:29 - paw - INFO - HOST: https://api.ibm.com/geospatial/run/na/core/v3


## Query to estimate usage

The following code snippet submits the query for the Data Layers 49360,49681 and 49689 corresponding to Band 4 (red),Band 3 (green) and Water vapor for an area in South of USA spanning approximately 10,000 hectares for the month of January 2023 using Geospatial APIs V3 query API with the dryRun enabled to estimate resource usage.

We will go ahead and submit the query and observe the output.

In [15]:
v3_query_object = query.submit({
                          "name": "usage calculation",
                          "dryRun": True,
                          "spatial": {
                            "type": "poly",
                            "polygon": {
                              "wkt": "Polygon ((-98.11179332181011148 28.64406107625320885, -98.12378016740014175 28.72704693033795209, -98.01589855708996879 28.72028512000512279, -97.99592048110660869 28.63668455589012041, -98.11179332181011148 28.64406107625320885))"
                            }
                          },
                          "temporal": {
                            "intervals": [
                              {
                                "start": "2023-01-01 00:00:00",
                                "end": "2023-12-31 00:00:00"
                              }
                            ]
                          },
                          "layers": [
                            {  
                             "type": "raster",
                              "id": "49360",
                              "output": False
                            },
                            {
                              "type": "raster",
                              "id": "49681",
                              "output": False
                            },
                            {
                              "type": "raster",
                              "id": "49689",
                              "output": False
                            }
                          ]
                        }, client=EI_client_v3)

print(v3_query_object.submit_response)

2025-03-20 13:37:33 - paw - INFO - TASK: submit STARTING.
2025-03-20 13:37:39 - paw - INFO - TASK: submit COMPLETED.
{
    "info": {
        "count": 0,
        "interactive_max_count": 0,
        "max_count": 0,
        "max_usage": 6442450.944,
        "usage": 25386552,
        "usage_unit": "bytes"
    }
}


### Understanding the output
The API response provides an estimated usage of 25386552 bytes per API call when querying the data for the spatial and temporal range mentioned above. Please note that the resource usage estimate increases with the expansion in temporal and spatial ranges.

Similarly, usage can be calculated for different spatial and temporal ranges of interest for various combinations of Data Layers, ensuring resource estimates tailored to specific query parameters.